In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# data에 저장된 etri-bert 모델 가져와서 압축해제
!cp "/content/gdrive/My Drive/data/1_bert_download_001_bert_morp_pytorch.zip" "1_bert_download_001_bert_morp_pytorch.zip"
!unzip "1_bert_download_001_bert_morp_pytorch.zip"

In [ ]:
!cp "/content/gdrive/My Drive/data/bert_data.zip" "bert_data.zip"
!unzip "bert_data.zip"

# 1. PYROUGE (ROGUE METRIC PACKAGE)와 의존성 패키지 설치

In [ ]:
# install bheinzerling's pyrouge
!git clone https://github.com/bheinzerling/pyrouge
%cd pyrouge
!python setup.py install
# install missing dependency
!apt install libxml-parser-perl
%cd pyrouge
!git clone https://github.com/andersjo/pyrouge.git rouge
!pyrouge_set_rouge_path '/content/pyrouge/pyrouge/rouge/tools/ROUGE-1.5.5'
%cd /content/pyrouge/pyrouge/rouge/tools/ROUGE-1.5.5/data
!mv WordNet-2.0.exc.db WordNet-2.0.exc.db.orig
!perl WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db

pyrouge 동작 Test (마지막 출력에 ok가 나오면 성공!)

In [ ]:
!python -m pyrouge.test

In [ ]:
# 기타 패키지 설치
!pip install pytorch_pretrained_bert
!pip install tensorboardX

# 2. GOOGLE DRIVE MOUNT

In [ ]:
# Bertsum directory chdir
import os
os.chdir('/content/gdrive/My Drive/packages/BertSum/src')

# Train

**Arguments**

-mode train (train, validate, test) # 학습 모드 설정

-encoder classifier (classifier, transformer, RNN) # 학습 방법 설정

-dropout 0.1 # dropout rate

-bert_data_path ../bert_data/korean # bert.pt파일 저장되어있는 경로(pt파일 앞의 단어까지 경로에 쓸 것)

-model_path ../models/bert_classifier_sample # train의 경우 학습된 모델 저장

-visible_gpus 0 # -1일 경우 cpu, 0이상일 경우 cuda 사용

In [ ]:
!python train.py -mode train -encoder classifier -dropout 0.1 -bert_data_path /content/bert_data/korean -model_path ../models/bert_classifier -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 -batch_size 1000 -decay_method noam -train_steps 1000 -accum_count 1 -log_file ../logs/bert_classifier -use_interval true -warmup_steps 8000 -bert_model /content/001_bert_morp_pytorch -bert_config_path /content/001_bert_morp_pytorch/bert_config.json -temp_dir .

In [ ]:
!python train.py -mode validate -bert_data_path /content/bert_data/korean -model_path ../models/bert_classifier  -visible_gpus 0  -gpu_ranks 0 -batch_size 1000  -log_file ../logs/bert_classifier  -result_path ../results/korean -bert_model /content/001_bert_morp_pytorch

In [ ]:
!python train.py -mode test -bert_data_path /content/bert_data/korean -model_path ../models/bert_classifier  -visible_gpus 0  -gpu_ranks 0 -batch_size 1000  -log_file ../logs/bert_classifier  -result_path ../results/korean -bert_model /content/001_bert_morp_pytorch -test_from ../models/bert_classifier/model_step_1000.pt

# Test

In [ ]:
with open('/content/gdrive/My Drive/packages/BertSum/results/korean_step1000.gold','r') as f:
    gold = f.readlines()

In [ ]:
with open('/content/gdrive/My Drive/packages/BertSum/results/korean_step1000.candidate','r') as f:
    candidate = f.readlines()

In [ ]:
# 라벨 데이터
gold[3]

In [ ]:
# 예측 데이터
candidate[3]